# **Ускорение Python**

In [1]:
import numpy as np

In [2]:
a = np.random.rand(2_000_000).astype(np.longdouble)
b = a.copy()
id(a) == id(b)

False

In [3]:
%%timeit -n 10

[ i*j for i,j in zip(a, b) ]

624 ms ± 22.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%%timeit -n 10
global a
a*=b

2.33 ms ± 169 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
from numba import float64, guvectorize

@guvectorize([(float64[:], float64[:])], '(a), (b)') 
#Можно подключить JIT-компилятор, но vectorize лучше оптимизирует вычислительный код
#@njit('void(f8[:], f8[:])')
def cross_pr(a, b):
    
    for i in range(a.shape[0]):
        a[i] = a[i] * b[i]

In [7]:
%timeit -n 10 cross_pr(a, b)

2.38 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
from aot_compiled import cross_pr_aout
#можно скомпилировать заранее, но скорость почти одинаковая.

In [9]:
%%timeit -n 10
cross_pr_aout(a,b)

2.69 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Как можно увидеть далее numpy типы не уступают в точности long double из си.  
https://ru.wikipedia.org/wiki/Число_двойной_точности

In [10]:
t = np.array([i for i in range(70_000)], dtype=np.float64)

try:
    ty = np.iinfo(t.dtype)
except:
    ty = np.finfo(t.dtype)
ty

finfo(resolution=1e-15, min=-1.7976931348623157e+308, max=1.7976931348623157e+308, dtype=float64)

In [11]:
np.finfo(np.float64) == np.finfo(np.longdouble)

True

# Итоги:

Python *медленный*, но есть стероиды для тяжелых вычислительных расчетов.    
**Numpy** для уже существующих матричных и векторных алгоритмов(базовые действия типа сложения, умножения и т.д., транспозиция...)  
**Numba** для реализации алгоритмов, которых в numpy нет